## Base de Dados
O objetivo final do projeto é usar uma base de dados com as relações de parentesco dentro do bairro de Surubeju do município de Monte Alegre - PA. No entanto, devido ao tempo de coleta desses dados, usarei um apenas uma amostra ou um conjunto de dados fake para a criação da visualização.

## Motivação
No referido bairro, no qual eu resido, existe uma peculiaridade de que os habitantes deste local tendem a constituir famílias com moradores do próprio bairro. Diante disso, criaram-se relações de parentesco peculiares, complexas e até mesmo desconhecidas pelos residentes. Com isso, decidi criar uma visualização dinâmica que demonstrasse todas as relações de parentesco presentes no bairro.

## Expectativa
O objetivo é conseguir plotar um gráfico de redes interativo, em que o usuário possa inserir o nome de duas pessoas e verificar a rede de parentesco entre elas.


## Esboços
Estão algumas amostras de dados com relações de parentesco e esboços de como será a visualização final.

In [2]:
import pandas as pd

import networkx as nx

import matplotlib.colors as mcolors

import dash
import dash_html_components as html
import dash_cytoscape as cyto
import dash_core_components as dcc
from dash.dependencies import Input, Output


C:\Users\LUIZLU~1\AppData\Local\Temp/ipykernel_9816/1395834552.py:8: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\LUIZLU~1\AppData\Local\Temp/ipykernel_9816/1395834552.py:10: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [3]:
# Coletando Data Frame do xls
full_data = pd.read_excel('familia.xlsx', index_col=0)
full_data.head()


,nome,apelido,pai_id,pai,mae_id,mae
id,,,,,,
1,Waldomiro Vasconcelos,NaN,NaN,NaN,NaN,NaN
2,Waldomira Vasconcelos,NaN,NaN,NaN,NaN,NaN
3,Waldomira Vasconcelos,Miroca,1.0,Waldomiro Vasconcelos,2.0,Waldomira Vasconcelos
4,Kelly Regina Vasconcelos,Branca,98.0,Pedro Bandeira,3.0,Waldomira Vasconcelos
5,Cláudio Vasconcelos,Bloco,98.0,Pedro Bandeira,3.0,Waldomira Vasconcelos


In [4]:
name_by_id = full_data[["nome"]].to_dict()["nome"]


In [5]:
# Contando o número de pessoas no banco de dados
num_people = len(full_data['nome'].unique())
# Criando uma lista de cores disponíveis
color_list = list(mcolors.cnames.keys())
# Criando uma lista de cores de acordo com o número de pessoas
chosen_colors = color_list[1:num_people+3]
# Dicionário das pessoas correspondentes a cada cor
colors_by_id = dict(zip(full_data['nome'].index, chosen_colors))
# Criando um dataframe com as pessoas e suas cores correspondentes.
# df_colors = pd.DataFrame({'classes':full_data['nome'].unique(), 'colors':chosen_colors})
# df_colors.head()


In [6]:
# Criando o Grafo
# Adicionando os nodes
family_tree = nx.DiGraph()

for idx in full_data.index:
    family_tree.add_node(str(idx),
                         id=str(idx),
                         label=name_by_id[idx],
                         color=colors_by_id[idx])


In [7]:
# Adicionando Edges
father_ids = full_data["pai_id"].values
mother_ids = full_data["mae_id"].values
name_ids = full_data["nome"].index.values
edges_list = []

for name_id, father_id, mother_id in zip(name_ids, father_ids, mother_ids):
    if pd.notna(father_id):
        edges_list.append((str(name_id), str(int(father_id))))
    if pd.notna(mother_id):
        edges_list.append((str(name_id), str(int(mother_id))))


family_tree.add_edges_from(edges_list)


In [12]:
cyto_family_nodes = nx.readwrite.json_graph.cytoscape_data(family_tree)[
    'elements']['nodes']
cyto_family_edges = nx.readwrite.json_graph.cytoscape_data(family_tree)[
    'elements']['edges']


In [42]:
family_degrees = family_tree.in_degree
for node in cyto_family_nodes:
    node["data"]["size"] = family_degrees[node["data"]["id"]]**2


In [147]:
def get_path(family_tree:nx.classes.digraph.DiGraph, source_id:str, target_id:str):
    source = nx.descendants(family_tree,source_id)
    source.add(source_id)
    target = nx.descendants(family_tree,target_id)    
    target.add(target_id)
    intersec_parent_nodes =target.intersection(source)
    shortest_path_length = float("inf")
    all_shortest_paths = []
    for parent in intersec_parent_nodes:
        path_length =nx.shortest_path_length(family_tree,source_id,parent)+nx.shortest_path_length(family_tree,target_id,parent)
        if path_length < shortest_path_length:
           all_shortest_paths.clear()

        if path_length <= shortest_path_length:
           source_to_parent = [path for paths in nx.all_shortest_paths(family_tree,source_id,parent) for path in paths]

           target_to_parent = [path for paths in nx.all_shortest_paths(family_tree,target_id,parent) for path in reversed(paths[:-1])]
           
           all_shortest_paths.append(source_to_parent+target_to_parent)

           shortest_path_length = path_length

    return all_shortest_paths
        



get_path(family_tree,"70","9")

[['70', '67', '2', '3', '7', '8', '9'], ['70', '67', '1', '3', '7', '8', '9']]

In [153]:
cyto.load_extra_layouts()

In [170]:
# Melhores layouts
# breadthfirst
# dagre
# klay
cytoscape_stylesheet = [
    {'selector': 'node',
     'style': {'label': 'data(label)',
               'padding': 'data(size)',
               'border-width': '1px',
               'border-style': 'solid',
               'border-color': "black"

               },

     "text-wrap": "wrap"},
    {'selector': 'edge',
     'style': {
         'line-color': "grey",
         'opacity': '0.5'
     }},
    # {
    #     "selector": 'node',
    #     "style": {
    #         "width": "data(size)px",
    #         "height": "data(size)px",
    #         "padding": "data(size)"
    #     }
    # }
]

app = dash.Dash("Family Network")
app.layout = html.Div([
    html.P('Dash Cytoscape:'),
    html.H2("Conheça a conexão entre duas pessoas do bairro"),
    html.Div([
        dcc.Dropdown(
            id='input-father-id',
            options=[{'label': name, 'value': idx}
                     for (idx, name) in name_by_id.items()],
            searchable=True,
            style={"margin": "5px", "width":"40vw"}
        ),
        dcc.Dropdown(
            id='input-mother-id',
            options=[{'label': name, 'value': idx}
                     for (idx, name) in name_by_id.items()],
            searchable=True,
            style={"margin": "5px", "width":"40vw"}
        ),
        ], style={"display":"flex",
                    "flex-direction":"row",
                    # "justify-content": "space-around",
                    }
        ),
    html.P( id="warning", style={"color":"red"}),
    cyto.Cytoscape(
        id='cytoscape',
        elements=cyto_family_nodes+cyto_family_edges,
        layout={'name': 'dagre',
                #  'roots':'#1, #2'
                },
        style={'width': '100vw', 'height': '90vh'},
        stylesheet=cytoscape_stylesheet)
])





@app.callback(Output('cytoscape', 'stylesheet'),
              Input('input-father-id', 'value'),
              Input('input-mother-id', 'value'))
def highlight_node_path(source_id, target_id):
    style = []
    if source_id is not None and target_id is not None:
        try:
            path = get_path(family_tree,str(source_id),str(target_id))
        except:
            pass
        edge_style = []
        node_style = []
        
        for node_path in path:
            
            node_style += [{"selector": f"#{idx}",
                        "style": {"background-color": "blue"}}
                        for idx in node_path]

        
            for num, idx in enumerate(node_path[:-1]):
                edge_style += [
                    {'selector': f"[target = '{node_path[num]}' ][source = '{node_path[num+1]}']",
                    'style': {
                        'line-color': "blue",
                    }},
                    {'selector': f"[source = '{node_path[num]}' ][target = '{node_path[num+1]}']",
                    'style': {
                        'line-color': "blue",
                    }},
                ]

        style = node_style + edge_style

    return cytoscape_stylesheet + style
    


app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Nov/2021 11:53:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2021 11:53:39] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2021 11:53:39] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2021 11:53:39] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2021 11:53:39] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2021 11:53:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2021 11:53:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2021 11:53:42] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
# @app.callback(Output('cytoscape', 'elements'),
#               Input('input-father-id', 'value'),
#               Input('input-mother-id', 'value'))
# def highlight_edge_path(source_id, target_id):
#     highlight_edges = []
#     print("Highlight edge path")
#     if source_id is not None and target_id is not None:
#         node_path = nx.dijkstra_path(family_tree.to_undirected(), source_id, target_id)
#         for num, idx in enumerate(node_path[:-1]):
#             highlight_edges += [{'data': {'source': node_path[num],
#                                             'target':node_path[num+1] ,
#                                             "classes":"highlight",
#                                             "class":"highlight",
#                                             "classname":"highlight",
#                                             "line-color":"red",
#                                             "color":"red"}
#                                             }]

#         return cyto_family_nodes+cyto_family_edges+highlight_edges
#     return cyto_family_nodes+cyto_family_edges


In [ ]:
# Informações importantes
# Usar seletores de metadados como degree, indegree ou outdegree


## Ideias para a visualização

- Destacar todos os parents de um node
- 

<img src='Grafo.png' width="400" height="400">